# Código do TCC de Paulo de Tarso, da EMAp FGV

In [307]:
import nltk
#Se for preciso, executar o comando abaixo
#nltk.download()
import unicodedata
import string
import os
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import CountVectorizer as CV
#Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

## Códigos auxiliares
### Código para remover símbolos das strings

In [308]:
def clean_string(s):
    s = s.replace("\n"," ")
    return ''.join(x for x in unicodedata.normalize('NFKD', s)
                   if x in string.ascii_letters + " ").lower()

## Parâmetros

In [309]:
#Parâmetros
inicio_atas = 104 #Utilizarei as atas a partir de 2005
final_atas = 205

total_atas = final_atas - inicio_atas + 1
atas = range(inicio_atas,final_atas+1)

path = "D:/Users/paulotarsosantos/Documents/Documentos/TCC/Atas/"

## Tokenização dos textos

#### Obter textos dos arquivos

In [310]:
texts = []

#Ir para a pasta com os arquivos
os.chdir(path)

#Obter o texto dos arquivos
for n in atas:
    file = open("COPOM_" + str(n) + ".txt")
    texts.append(clean_string(file.read()))
    file.close()
    
#Voltar uma pasta
os.chdir("..")

#### Vetorizar os textos obtidos

#### Stopwords

In [311]:
sw = [clean_string(word) for word in 
             open("stopwords.txt",encoding="utf-8").read().splitlines()[1:]]

#### Vectorização

In [312]:
vectorizer = CV(stop_words=sw,min_df=50)
arrays = vectorizer.fit_transform(texts).toarray()
#d_arrays = dict(zip(range(inicio_atas,final_atas+1),arrays))

## Obter os valores das variações na meta da Selic em cada reunião do Copom

In [313]:
file = open("Cortes.txt")
cortes = [int(line.split(";")[2]) for line in file.read().splitlines()[1:]]
cortes = [int(n/abs(n)) if n != 0 else 0 for n in cortes]
cortes.reverse()
file.close()

## Métodos de classificação

### GLM

In [314]:
from sklearn.linear_model import LinearRegression as LR
classifier = LR()
n = 10000
score = 0
last_prediction = 0
for i in range(n):
    #Separação em dados para treino e dados para teste do modelo
    X_train,X_test,y_train,y_test = tts(arrays[:-1],cortes[:-1],train_size=0.9)
    
    classifier.fit(X_train,y_train)
    
    score += classifier.score(X_test,y_test)
    last_prediction += classifier.predict(arrays[-1])[0]
score /= n
last_prediction /= n
print(score,last_prediction)

0.519663119083 -0.794516759239


In [315]:
total_atas

102

In [316]:
classifier.predict(X_test),y_test

(array([-1.62188524,  0.14002572, -1.13365075, -1.01922298, -0.12234857,
        -0.19225777, -1.03604204, -0.72802043, -0.32622194,  0.75284272,
        -1.04524001]), [-1, 0, -1, -1, 1, 1, -1, -1, -1, 1, 0])

### SVM

In [317]:
from sklearn.svm import SVC
classifier = SVC()
n = 10000
score = 0
last_prediction = 0
for i in range(n):
    #Separação em dados para treino e dados para teste do modelo
    X_train,X_test,y_train,y_test = tts(arrays[:-1],cortes[:-1],train_size=0.9)
    
    classifier.fit(X_train,y_train)
    
    score += classifier.score(X_test,y_test)
    last_prediction += classifier.predict(arrays[-1])[0]
score /= n
last_prediction /= n
print(score,last_prediction)

0.383336363636 -1.0


In [318]:
classifier.predict(X_test),y_test

(array([-1, -1,  1,  0, -1,  1, -1, -1, -1, -1, -1]),
 [0, 0, 0, 1, -1, 0, 1, 1, 0, -1, 0])